In [1]:
import torch
import os, sys
import numpy as np
parent_dir = os.path.abspath('../')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
os.environ['CUDA_VISIBLE_DEVICES']='0'
from torch.nn.functional import pad,fold
from util.util import kbdwin

In [ ]:
a = torch.arange(start=0, end=16,step=1, dtype=torch.float64)
exp = torch.exp(-2j*torch.pi/16*(torch.arange(start=0, end=16//4,step=1, dtype=torch.float64)+1/8))
print(exp)
a = (a[...,:16//2:2]-a[...,1+16//2::2].flip(-1)) - 1j*(a[...,16//2::2]-a[...,1:16//2:2].flip(-1))
print(a)
a = a*exp
a = torch.fft.fft(a)
a = a*exp
print(a)
a = 2/torch.sqrt(torch.tensor(16))*torch.view_as_real(a)
print(a)
a[...,1] = -a[...,1].flip(-1)
a = a.flatten(-2)
print(a)


In [ ]:
a = torch.arange(start=0, end=16,step=1, dtype=torch.float64)
print(a[...,:16//2:2])
print(a[...,1:16//2:2])
print(a[...,16//2::2])
print(a[...,1+16//2::2])
b = torch.tensor([[1,2,3,4],[5,6,7,8]])
print(b.transpose(-1,-2).flatten(-2))

In [ ]:
a = torch.arange(start=0, end=16,step=1, dtype=torch.float64)
print(a)
a = a.unfold(-1,2,2)
print(a)
a[...,1] = a[...,1].flip(-1)
print(a)
a = a.unfold(-2,4,4)
print(a)
#a = a[...,0]-a[...,1] + 1j*(a[...,2]-a[...,3])
print(a[0,0,:])


In [ ]:
c = torch.stack((torch.arange(start=32//2-1, end=0, step=-2, dtype=torch.long),torch.arange(start=0, end=32//2, step=2, dtype=torch.long)),dim=0)
print(c.shape)
print(c.flatten(-2))

In [ ]:
b = torch.tensor([4,5,6,7])
a = torch.tensor([0,1,2,3])
c = torch.stack((a,b),-1).T.flatten(-2)
d =  torch.stack((c,-c.flip(-1)),dim=-1).flatten(-2)
print(d)

In [ ]:
a = torch.arange(start=0, end=16,step=1, dtype=torch.float64)
# a = a.unfold(-1,2,2)
# a.transpose(-1,-2).flatten(-2).roll(8)
# a[...,-1] = a[...,-1].flip(-1)
# a = a.unfold(0,4,4)
print(a.roll(4,dims=-1))

In [ ]:
a = torch.tensor([[1,2,3,4],[5,6,7,8]])
a.flatten(-2)

In [28]:
from torch_scatter import scatter
from typing import Optional, Union, Callable
from einops import rearrange
import torch.autograd.profiler as profiler

class FastMDCT4(torch.nn.Module):
    def __init__(self, n_fft: Optional[int] = 2048, hop_length: Optional[int] = None, win_length: Optional[int] = None, window: Union[torch.Tensor, np.ndarray, list, Callable, None] = None, center: bool = True, pad_mode: str = 'constant', device: str = 'cuda') -> None:
        super().__init__()
        self.n_fft = n_fft
        self.pad_mode = pad_mode
        self.device = device
        self.hop_length = hop_length
        self.center = center

        if callable(window):
            self.win_length = int(win_length)
            self.window = window(self.win_length).to(
                device=self.device, dtype=torch.float64)
        elif isinstance(window, torch.Tensor):
            self.window = window.to(device=self.device, dtype=torch.float64)
            self.win_length = len(window)
        elif isinstance(window, np.ndarray) or isinstance(window, list):
            self.window = torch.tensor(
                window, device=self.device, dtype=torch.float64)
            self.win_length = len(window)
        elif window is None:
            if win_length is not None:
                self.win_length = win_length
            elif n_fft is not None:
                self.win_length = n_fft
            else:
                assert False, 'You should specify window length or n_fft'
            self.window = torch.ones(
                (self.win_length,), device=self.device, dtype=torch.float64)
        else:
            raise NotImplementedError

        assert self.win_length <= self.n_fft, f'Window lenth {self.win_length} should be no more than fft length {self.n_fft}'
        assert self.hop_length <= self.win_length, 'You hopped more than one frame'

        self.idx = torch.stack((
            torch.arange(
                start=0, end=n_fft//2, step=2,
                dtype=torch.long, device=self.device),
            torch.arange(
                start=n_fft-1, end=n_fft//2, step=-2,
                dtype=torch.long, device=self.device),
            torch.arange(
                start=n_fft//2, end=n_fft, step=2,
                dtype=torch.long, device=self.device),
            torch.arange(
                start=n_fft//2-1, end=0, step=-2,
                dtype=torch.long, device=self.device)
        ), dim=0)

        # self.sqrtN = torch.sqrt(torch.tensor(
        #     [self.n_fft], device=self.device, dtype=torch.float64))
        self.post_exp = torch.exp(
            -2j*torch.pi/self.n_fft*(
                torch.arange(
                    start=0,
                    end=self.n_fft//4,
                    step=1,
                    dtype=torch.float64,
                    device=self.device
                )+1/8
            )
        )

        self.pre_exp = (self.make_pre_exp()*self.window).to(torch.complex64)
        self.post_exp = self.post_exp.to(torch.complex64)
        # self.pre_idx = self.make_pre_idx()
        self.post_idx = self.make_post_idx()
        self.idx = self.idx.clone().mT.roll(self.n_fft//8,0).contiguous()
        self.kernel = self.pre_exp[..., self.idx].permute(1,0,2).contiguous()

    def make_pre_exp(self):
        sgn = torch.ones(1, self.n_fft, dtype=torch.complex128,
                         device=self.device)
        # Shift for Time-Domain Aliasing Cancellation (TDAC)
        sgn[..., -self.n_fft//4:] *= -1
        sgn = sgn.roll(self.n_fft//4, dims=-1)
        sgn[..., self.idx[0]] *= self.post_exp
        sgn[..., self.idx[1]] *= -self.post_exp
        sgn[..., self.idx[2]] *= -1j*self.post_exp
        sgn[..., self.idx[3]] *= 1j*self.post_exp
        return sgn.roll(-self.n_fft//4, dims=-1).to(self.device).contiguous()

    # def make_pre_idx(self):
    #     i = torch.arange(start=0, end=self.n_fft, step=1,
    #                      dtype=torch.long, device=self.device)
    #     i = i.roll(self.n_fft//4, dims=-1)
    #     idx_ = torch.stack([i[self.idx[0]], i[self.idx[1]],
    #                        i[self.idx[2]], i[self.idx[3]]], dim=1)
    #     index = torch.zeros(
    #         1, self.n_fft, device=self.device, dtype=torch.long)
    #     for i in torch.arange(0, self.n_fft//4, dtype=torch.long):
    #       index[..., idx_[i]] = i
    #     return index.squeeze().contiguous()

    def make_post_idx(self):
        idx = torch.arange(self.n_fft//2, dtype=torch.long,
                           device=self.device).reshape(-1, 2)
        idx[:, 1] = idx[:, 1].flip(-1)
        return idx.flatten()

    def forward(self, signal: torch.tensor, return_frames: bool = False):
        if signal.dim() == 2: # B T (mono)
            signal = signal[:, None, :]
        elif signal.dim() == 3: # B C T (stereo)
            pass
        else:
            raise NotImplementedError

        # Pad the signal to a proper length
        B, C, T = signal.shape
        start_pad = 0
        # Pad the signal so that the t-th frame is centered at time t * hop_length. Otherwise, the t-th frame begins at time t * hop_length.
        if self.center:
            start_pad = self.hop_length
        additional_len = T % self.hop_length
        end_pad = start_pad
        if additional_len:
            end_pad = start_pad + self.hop_length - additional_len
        signal = pad(signal, (start_pad, end_pad), mode=self.pad_mode)

        # Slice the signal with overlapping
        signal = signal.unfold(dimension=-1, size=self.win_length, step=self.hop_length)
        # B C T -> B C N t
        B,C,N,t = signal.shape
        # signal_1 = signal.clone()*self.pre_exp
        # signal_1 = scatter(signal_1, self.pre_idx, dim=-1, reduce='sum')

        # Black magik here: fully exploiting the symmetric property of O2FFT,
        # by rearranging the original real sequence into
        # the real and imag part of a new sequence with half of the length.
        # pre-twiddle
        # signal = signal[..., self.idx]
        signal = torch.gather(signal[...,None].expand(-1, -1, -1, -1, self.n_fft//4), -2, self.idx[None, None, None,...].expand(B, C, N, -1, -1))
        signal = rearrange(signal, 'B C N t n -> (B C N) t n')
        signal = torch.nn.functional.conv1d(
            input=signal.to(torch.complex64),
            weight=self.kernel,
            stride=1,
            groups=self.n_fft//4
        )
        signal = torch.fft.fft(signal[...,0], dim=-1)
        
        # post-twiddle
        signal = torch.conj_physical(signal*self.post_exp)
        # rearranging
        signal = torch.view_as_real(signal)
        signal = signal.view(B*C*N, -1)[..., self.post_idx]
        signal = rearrange(signal, '(B C N) t -> B C N t', B=B, C=C)

        return signal, None


In [ ]:
fmdct = FastMDCT4(n_fft=32, hop_length=32//2, win_length=32, window=None, center=True, device='cuda')
# print(fmdct.idx)
# print(fmdct.pre_idx)

In [ ]:
idx = fmdct.idx.cpu()
print(idx)

In [ ]:
signal = torch.arange(0,32).float().repeat(2,1)
print(signal.shape)
exp = torch.ones(1,32)*torch.tensor([1,2,3,4]).repeat(8)
signal *= exp.float()
print(signal)
b = torch.gather(signal[...,None].expand(-1, -1 ,8), -2, idx[None,...].expand(2,-1,-1))
print(b)
print(b.shape)

In [ ]:
post_idx = fmdct.post_idx.cpu()
signal = post_idx.clone().reshape(-1,2).float()
print(post_idx)
o = torch.zeros((1,16)).float().scatter_(1, post_idx[None,...], signal.view(1,-1))
print(o)

In [ ]:
signal = torch.arange(0,32).repeat(2,1)
exp = torch.ones(1,32)*torch.tensor([1,2,3,4]).repeat(8)
print(signal)
print(exp.shape)
signal = scatter(signal*exp, fmdct.pre_idx.cpu(), dim=-1, reduce='sum')
print(signal)

In [ ]:
signal = torch.arange(0,32).repeat(2,1)
idx = fmdct.idx.mT.cpu().roll(4,0)
print(idx.shape)
k = exp[...,idx].permute(1,0,2)
print(k.shape)
o = torch.nn.functional.conv1d(signal[...,idx].float(), k, stride=1, groups=8)
# o = o[:,torch.LongTensor([4, 3, 5, 2, 6, 1, 7, 0]),:]
#                           4  5  6  7  0  1  2  3
print(o)
print(o.shape)

In [47]:
class FastIMDCT4(torch.nn.Module):
    def __init__(self, n_fft: Optional[int] = 2048, hop_length: Optional[int] = None, win_length: Optional[int] = None, window: Union[torch.Tensor, np.ndarray, list, Callable, None] = None, center: bool = True, pad_mode: str = 'constant', out_length: Optional[int] = None, device: str = 'cuda') -> None:
        super().__init__()
        self.n_fft = n_fft
        self.pad_mode = pad_mode
        self.device = device
        self.hop_length = hop_length
        self.center = center
        self.out_length = out_length

        if callable(window):
            self.win_length = int(win_length)
            self.window = window(self.win_length).to(
                device=self.device, dtype=torch.float64)
        elif isinstance(window, torch.Tensor):
            self.window = window.to(device=self.device, dtype=torch.float64)
            self.win_length = len(window)
        elif isinstance(window, np.ndarray) or isinstance(window, list):
            self.window = torch.tensor(
                window, device=self.device, dtype=torch.float64)
            self.win_length = len(window)
        elif isinstance(window, None):
            if win_length is not None:
                self.win_length = win_length
            elif n_fft is not None:
                self.win_length = n_fft
            else:
                assert False, 'You should specify window length or n_fft'
            self.window = torch.ones(
                (self.win_length,), device=self.device, dtype=torch.float64)
        else:
            raise NotImplementedError

        assert self.win_length <= self.n_fft, f'Window lenth {self.win_length} should be no more than fft length {self.n_fft}'
        assert self.hop_length <= self.win_length, 'You hopped more than one frame'

        self.exp = torch.exp(
            -2j*torch.pi/self.n_fft*(
                torch.arange(
                    start=0,
                    end=self.n_fft//4,
                    step=1,
                    dtype=torch.float32,
                    device=self.device
                )+1/8
            )
        ).contiguous()
        self.pre_idx = self.make_pre_idx()
        self.post_idx = self.make_post_index()
        #self.sgn = self.make_sign()
        self.window = (4.0*self.make_sign()*self.window/self.n_fft).to(torch.float32).contiguous()

    def make_pre_idx(self):
        a = torch.arange(self.n_fft//2, dtype=torch.long,
                         device=self.device).unfold(-1, 2, 2)
        return torch.stack((a[..., 0], a[..., 1].flip(-1)), dim=-1).contiguous()

    def make_post_index(self):
        a = torch.arange(0, self.n_fft//2, 2,
                         dtype=torch.long, device=self.device)
        b = torch.arange(self.n_fft//2-1, 0, -2,
                         dtype=torch.long, device=self.device)
        idx = torch.empty((self.n_fft,), dtype=torch.long, device=self.device)
        idx[0:self.n_fft//2:2] = a
        idx[1:self.n_fft//2:2] = b
        idx[self.n_fft//2:] = idx[:self.n_fft//2].flip(0)
        return idx.roll(-self.n_fft//4).contiguous()

    def make_sign(self):
        sign = torch.ones((self.n_fft,), device=self.device,
                          dtype=torch.float64)
        sign[1::2] *= -1
        sign[..., 0:self.n_fft//4] *= -1
        return sign.roll(-self.n_fft//4).contiguous()

    def forward(self, signal: torch.Tensor, return_frames: bool = False):
        assert signal.dim(
        ) <= 4, f'Only tensors shaped in BHW or BCHW are supported, got tensor of shape {signal.shape}'
        assert signal.shape[
            -1] == self.n_fft//2, f'The last dim of input tensor should match the n_fft. Expected {self.n_fft}, got {signal.shape[-1]}'

        if signal.dim() == 4:
            C = signal.shape[1]
            signal = rearrange(signal, 'B C T N -> (B C) T N')
        else:
            C = 1

        signal = signal.to(self.device)
        # # Inverse transform at the last dim
        signal = torch.view_as_complex(signal[..., self.pre_idx])

        signal = self.exp*signal
        signal = torch.fft.fft(signal)
        signal = self.exp*signal

        # [0+4j, 1+5j, 2+6j, 3+7j] -> [2,-5, 3, -4, 4, -3, 5, -2, 6, -1, 7, 0, 0, 7, -1, 6]
        signal = torch.view_as_real(signal).flatten(-2)[..., self.post_idx]

        # Apply windows to each pieces
        signal = self.window*signal
        if return_frames:
            frames = signal.clone()
        else:
            frames = torch.empty(1)

        # Overlapping adding by fold()
        out_len = (signal.shape[-2]-1) * self.hop_length + self.win_length
        signal = fold(signal.mT, kernel_size=(1, self.win_length),
                      stride=(1, self.hop_length), output_size=(1, out_len))

        if self.center:  # extract the middle part
            signal = signal[..., self.win_length//2:-self.win_length//2]
        if self.out_length is not None:
            signal = signal[..., :self.out_length]
        # signal = 4.0*signal / (self.n_fft)
        if C != 1:
            signal = rearrange(signal, '(B C) T N-> B C T N')
        return signal, frames


In [ ]:
b = torch.arange(0,32,2)
print(b)
a = torch.arange(16).unfold(-1,2,2)
a = torch.stack((a[..., 0], a[...,1].flip(-1)), dim=-1)
print(b[a])

In [ ]:
a = torch.tensor([0+4j, 1+5j, 2+6j, 3+7j])
a = torch.view_as_real(a).flatten()
print(a)
# [0, -7,  1, -6,  2, -5,  3, -4,  4, -3,  5, -2,  6, -1,  7,  0]
#b = torch.tensor([[0,15],[7,8],[2,13],[5,10],[4,11],[3,12],[6,9],[1,14]])
b = torch.LongTensor([0,7,2,5,4,3,6,1,1,6,3,4,5,2,7,0])
c = a[b].roll(-4)
print(c)

In [ ]:
n_fft = 16
a = torch.arange(0,n_fft//2,2)
b = torch.arange(n_fft//2-1,0,-2)
c = torch.empty((n_fft,))
c[0:n_fft//2:2] = a
c[1:n_fft//2:2] = b
c[n_fft//2:] = c[:n_fft//2].flip(0)
c = c.roll(-n_fft//4)
c = c.to(torch.long)

In [ ]:
a = torch.tensor([0+4j, 1+5j, 2+6j, 3+7j])
a = torch.view_as_real(a).flatten()
print(a[c])

In [ ]:
a = torch.tensor([0, -7,  1, -6,  2, -5,  3, -4,  4, -3,  5, -2,  6, -1,  7,  0])
a[...,0:n_fft//4] *= -1
a = a.roll(-n_fft//4,dims=-1)
print(a)

In [ ]:
b = torch.ones((n_fft))
b[1::2] *= -1
b[...,0:n_fft//4] *= -1
b = b.roll(-n_fft//4)
print(a[c]*b)

In [3]:
N = 512
win=kbdwin(N).cuda()
# mdct = FastMDCT4(n_fft=N, hop_length=N//2, win_length=N, window=win, center=True, device='cuda')
# imdct = FastIMDCT4(n_fft=N, hop_length=N//2, win_length=N, window=win, center=True, device='cuda')

In [ ]:
import torchaudio
import matplotlib.pyplot as plt
import torch
import torchaudio.functional as aF
audio_path = '/home/neoncloud/pix2pixHDAudioSR/test/test.wav'
metadata = torchaudio.info(audio_path)
print(metadata.num_frames)
audio, fs = torchaudio.load(audio_path)
K = 50000
audio_seg = audio[...,K:130816+K].squeeze()
print(audio_seg.shape)
plt.plot(audio_seg)
plt.show()

In [ ]:
audio_seg = torch.stack([audio_seg]*16)
spectro, _ = fast_mdct(audio_seg.cuda().squeeze(0))
# spectro_ = aF.amplitude_to_DB((spectro.abs().permute(1,0) + 1e-4),20,1e-4,1).squeeze(0)
print(spectro.size())
print(spectro.device)
sp_fig, sp_ax = plt.subplots()
sp_ax.pcolormesh(spectro[0].cpu().numpy(), cmap='PuBu_r')
plt.show()

In [ ]:
reconstruction, _ = imdct(spectro)
err = (audio_seg.cpu().squeeze()[:130000] - reconstruction.squeeze().cpu()[:130000])**2
print(reconstruction.mean())
print(err.mean())
plt.plot(reconstruction.mean(0).cpu().squeeze())
plt.show()

In [ ]:
reconstruction, _ = fast_imdct(spectro)
err = (audio_seg.cpu().squeeze()[:,:130000] - reconstruction.squeeze().cpu()[:,:130000])
print(reconstruction.mean())
print(err.mean())
plt.plot(err.mean(1).cpu().squeeze())
plt.show()

In [ ]:
from models.mdct import IMDCT4
imdct_ = IMDCT4(n_fft=N, hop_length=N//2, win_length=N, window=win, center=True, device='cuda')

In [ ]:
reconstruction_, _ = imdct_(spectro.unsqueeze(0))
err = (audio_seg.cpu().squeeze()[:130000] - reconstruction_.squeeze().cpu()[:130000])**2
print(reconstruction_.mean())
print(err.mean())
plt.plot(reconstruction_.cpu().squeeze())
plt.show()

In [29]:
import timeit
from models.mdct import MDCT4, IMDCT4
N = 512
mdct = MDCT4(n_fft=N, hop_length=N//2, win_length=N, window=win, center=True, device='cuda')
fast_mdct = FastMDCT4(n_fft=N, hop_length=N//2, win_length=N, window=win, center=True, device='cuda')
torch.backends.cudnn.benchmark = True
#

In [ ]:
a = torch.randn(64,32512).cuda()
sig_1, sig_2 = fast_mdct(a)

In [ ]:
sig_1.squeeze_()
sig_2.squeeze_()
(sig_2-sig_1).pow(2).sum()

In [ ]:
print(fast_mdct.pre_idx.shape)
print(fast_mdct.pre_idx)

In [ ]:
print(fast_mdct.idx)

In [ ]:
%timeit -r 20 -n 500 mdct(torch.randn(64,32512, device='cuda'))

In [ ]:
%timeit -r 20 -n 500 fast_mdct(torch.randn(64,32512, device='cuda'))

In [33]:
a = torch.randn(64,32512).cuda()
A,_ = mdct(a)
B,_ = fast_mdct(a)
B.squeeze_()
(A-B).pow(2).sum()

tensor(7.9642e-06, device='cuda:0', dtype=torch.float64)

In [48]:
imdct = IMDCT4(n_fft=N, hop_length=N//2, win_length=N, window=win, center=True, device='cuda')
fast_imdct = FastIMDCT4(n_fft=N, hop_length=N//2, win_length=N, window=win, center=True, device='cuda')

In [49]:
a_, _ = imdct(A.squeeze())
b_, _ = fast_imdct(A.squeeze())
print((b_-a_).pow(2).sum())

tensor(6.5446e-08, device='cuda:0', dtype=torch.float64)


In [ ]:
a = torch.randn(64,32512).cuda()
k = 0
for i in range(a.shape[0]):
    A,_ = mdct(a[None,i,:].clone())
    a_,_ = fast_imdct(A)
    k += (a[i,:]-a_).pow(2)
print(k.sum())

In [ ]:
a = torch.randn(64,32512).cuda()
A,_ = mdct(a)
a_,_ = fast_imdct(A)
(a-a_).sum()

In [ ]:
%timeit -r 20 -n 500 imdct(torch.randn(64,128,256, device='cuda'))

In [ ]:
%timeit -r 20 -n 500 fast_imdct(torch.randn(64,128,256, device='cuda'))

In [39]:
a = torch.randn(64,32512).cuda()
fast_mdct(a)
a = torch.randn(64,32512).cuda()
with profiler.profile(with_stack=False, profile_memory=True) as prof:
    A, _ = fast_mdct(a)

In [40]:
print(prof.key_averages(group_by_stack_n=10).table(sort_by='self_cpu_time_total', row_limit=10))

---------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                       Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
---------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
           cudaLaunchKernel        23.32%     330.000us        23.32%     330.000us      15.714us           0 b           0 b           0 b           0 b            21  
                aten::empty         6.22%      88.000us         6.22%      88.000us       8.000us           0 b           0 b      56.07 Mb      56.07 Mb            11  
                aten::copy_         5.37%      76.000us        10.88%     154.000us      25.667us           0 b           0 b           0 b          

In [50]:
a = torch.randn(64,32512).cuda()
A, _ = fast_mdct(a)
a_ , _ = fast_imdct(A)
with profiler.profile(with_stack=False, profile_memory=True) as prof2:
    a_ , _ = fast_imdct(A)

In [51]:
print(prof2.key_averages(group_by_stack_n=10).table(sort_by='self_cpu_time_total', row_limit=10))

-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                 aten::to        33.94%     298.000us        33.94%     298.000us     298.000us           0 b           0 b           0 b           0 b             1  
         cudaLaunchKernel        16.74%     147.000us        16.74%     147.000us      18.375us           0 b           0 b           0 b           0 b             8  
              aten::index         9.91%      87.000us        20.27%     178.000us      89.000us           0 b           0 b      24.00 Mb      24.00 Mb        